In [8]:
import pandas as pd
import datetime

In [9]:
df1 = pd.read_excel("../Data/DDW_PCA0000_2011_Indiastatedist.xlsx", sheet_name='Sheet1', usecols = ['State', 'Level', 'Name', 'TRU', 'TOT_P', 'TOT_M', 'TOT_F'])
df1 = df1[df1["TRU"] == "Total"]
#Replacing State codes with state names in census data
for idx, item in df1[df1["Level"] == "STATE"].iterrows():
    df1.replace(item["State"], item["Name"], inplace = True)
df1 = df1[df1["Level"] == "STATE"]
df1.reset_index(inplace = True)
#Updating State names in census data according to vaccine data
df1.replace({'&': 'AND'}, regex=True, inplace = True)
df1.replace({'NCT OF DELHI': 'DELHI'}, regex=True, inplace = True)
df1.replace({'DAMAN AND DIU': 'Dadra and Nagar Haveli and Daman and Diu'.upper()}, regex=True, inplace = True)
df1.replace('DADRA AND NAGAR HAVELI', 'Dadra and Nagar Haveli and Daman and Diu'.upper(), inplace = True)
df1 = df1.groupby(['State', 'Level', 'Name'], as_index=False).agg('sum')

In [10]:
df2 = pd.read_csv("../Data/cowin_vaccine_data_districtwise.csv", skiprows = [1])
df2 = df2.fillna(0)
df2 = df2.groupby(['State_Code', 'State', 'District_Key', 'District'], as_index=False).agg('sum')
df2 = df2.apply(lambda x: x.astype(str).str.upper())     #to make every state in uppercase

In [14]:
state = {"stateid" : [], "populationleft" : [], "rateofvaccination" : [], "date" : []}

for idx, row in df1.iterrows():
    edate = datetime.datetime(2021, 8, 14)
    lweek = edate - datetime.timedelta(days=7)       #end of the 2nd last week
    try:
        scode = df2.State_Code[df2["State"] == row["Name"]].astype('category').values[0]
        state["stateid"].append(scode)
    except IndexError:
        continue
    
    n1 = list(df2[df2["State"] == row["Name"]][edate.strftime("%d-%m-%Y.3")].astype('category').values)
    n1 = [int(i) for i in n1]
    n1 = sum(n1)                 #total no of people with atleast 1 dose at end of 14-Aug-2021
    
    n2 = list(df2[df2["State"] == row["Name"]][lweek.strftime("%d-%m-%Y.3")].astype('category').values)
    n2 = [int(i) for i in n2]
    n2 = sum(n2)                 #total no of people with atleast 1 dose at end of 7-Aug-2021
    
    pleft = row["TOT_P"] - n1
    vacRate = n1 - n2
    weeks = -(-pleft // vacRate)      #taking ceil while calculating remaining no of weeks for vaccination completion
    date = edate + datetime.timedelta(days=(weeks * 7))
    
    state["populationleft"].append(pleft)
    state["rateofvaccination"].append(vacRate)
    state["date"].append(date)
    
    
    

In [15]:
df = pd.DataFrame(state)
df.to_csv("../Final Outputs/vaccinated-dose-ratio-state.csv", index = False)